In [1]:
srand(0)
using StatsBase
using DataFrames
using EconUtils
using VarianceCovarianceEstimators

In [2]:
df = DataFrame(PID = repeat(1:10, inner = 10), TID = repeat(1:10, outer = 10),
    X1 = rand(100), X2 = rand(100), X3 = rand(100))
df[:y] = (Matrix(df[[:X1, :X2, :X3]]) * ones(3,1) .+ repeat(rand(10), inner = 10) .+ repeat(rand(10), outer = 10) .+ rand(100))[:]
head(df)
X = hcat(ones(100), Matrix(df[[:X1, :X2, :X3]]))
writedlm("X.txt", X[:,2:end])
y = Vector(df[:y])
writedlm("y.txt", y)
β = X \ y
ŷ = X * β
û = y - ŷ;

In [3]:
mutable struct MyModel <: StatsBase.RegressionModel
    mm::Matrix{Float64}
    û::Vector{Float64}
end

In [4]:
model = MyModel(X, û)
StatsBase.modelmatrix(obj::MyModel) = getfield(obj, :mm)
StatsBase.residuals(obj::MyModel) = getfield(obj, :û)
StatsBase.dof_residual(obj::MyModel) = reduce(-, size(StatsBase.modelmatrix(obj)))
StatsBase.deviance(obj::MyModel) = StatsBase.residuals(obj).'StatsBase.residuals(obj) / StatsBase.dof_residual(obj)
StatsBase.nobs(obj::MyModel) = length(StatsBase.residuals(obj))

In [5]:
## Additional methods that allow for weights and penalized regression
## StatsBase.modelmatrix and StatsBase.residuals could be weighted
# function bread(obj::StatsBase.RegressionModel)
#     mm = StatsBase.modelmatrix(obj)
#     output = inv(cholfact!(mm.'mm)) # Could be adapted to inv(cholfact!(mm.'mm + Γ)) for Ridge Regression
#     return output
# end

In [6]:
## Method for cluster robust versions
# clusters(obj::StatsBase.RegressionModel) = Vector{Vector{Int64}}() # Default
## Modify clusters(obj::StatsBase.RegressionModel) to obtain a Clusters[Dimension[Levels[Observations]]]

In [7]:
groups = EconUtils.makegroups(df[[:PID, :TID]])

2-element Array{Array{Array{Int64,1},1},1}:
 Array{Int64,1}[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [11, 12, 13, 14, 15, 16, 17, 18, 19, 20], [21, 22, 23, 24, 25, 26, 27, 28, 29, 30], [31, 32, 33, 34, 35, 36, 37, 38, 39, 40], [41, 42, 43, 44, 45, 46, 47, 48, 49, 50], [51, 52, 53, 54, 55, 56, 57, 58, 59, 60], [61, 62, 63, 64, 65, 66, 67, 68, 69, 70], [71, 72, 73, 74, 75, 76, 77, 78, 79, 80], [81, 82, 83, 84, 85, 86, 87, 88, 89, 90], [91, 92, 93, 94, 95, 96, 97, 98, 99, 100]]
 Array{Int64,1}[[1, 11, 21, 31, 41, 51, 61, 71, 81, 91], [2, 12, 22, 32, 42, 52, 62, 72, 82, 92], [3, 13, 23, 33, 43, 53, 63, 73, 83, 93], [4, 14, 24, 34, 44, 54, 64, 74, 84, 94], [5, 15, 25, 35, 45, 55, 65, 75, 85, 95], [6, 16, 26, 36, 46, 56, 66, 76, 86, 96], [7, 17, 27, 37, 47, 57, 67, 77, 87, 97], [8, 18, 28, 38, 48, 58, 68, 78, 88, 98], [9, 19, 29, 39, 49, 59, 69, 79, 89, 99], [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]]

In [8]:
# VarianceCovarianceEstimators.clusters(obj::MyModel) = Vector{Vector{Int64}}()

In [9]:
vcov(model, :OLS)

4×4 Array{Float64,2}:
  0.0194882  -0.0108421    -0.0137318   -0.012001   
 -0.0108421   0.0179593     0.00259367   0.000940701
 -0.0137318   0.00259367    0.0216219    0.00371975 
 -0.012001    0.000940701   0.00371975   0.0207911  

In [10]:
vcov(model, :HC1)

4×4 Array{Float64,2}:
  0.0225953  -0.0132238   -0.0168062  -0.0130633 
 -0.0132238   0.0183607    0.0033488   0.00451041
 -0.0168062   0.0033488    0.0256802   0.0055025 
 -0.0130633   0.00451041   0.0055025   0.0178279 

In [11]:
vcov(model, :HC2)

4×4 Array{Float64,2}:
  0.0228941  -0.0133848   -0.0170258   -0.0132469 
 -0.0133848   0.0185044    0.00341252   0.00458796
 -0.0170258   0.00341252   0.0259233    0.00560879
 -0.0132469   0.00458796   0.00560879   0.0180105 

In [12]:
vcov(model, :HC3)

4×4 Array{Float64,2}:
  0.02417    -0.0141159   -0.0179722   -0.0139963 
 -0.0141159   0.0194303    0.00362422   0.00486252
 -0.0179722   0.00362422   0.0272652    0.0059569 
 -0.0139963   0.00486252   0.0059569    0.0189572 

In [13]:
VarianceCovarianceEstimators.clusters(obj::MyModel) = groups

In [14]:
## Cluster robust estimates include the dof_residual for inference based on the cluster structure

In [15]:
vcov(model, :OLS)

LoadError: [91mAssertionError: OLS is not a valid cluster robust variance covariance estimator. Valid estimators are: HC1, HC2, HC3.[39m

In [16]:
V, rdf = vcov(model, :HC1)
V

4×4 Array{Float64,2}:
  0.0397154   -0.0185192   -0.00983352  -0.0419261
 -0.0185192    0.0220154   -0.00520504   0.0261645
 -0.00983352  -0.00520504   0.0233224    0.0127629
 -0.0419261    0.0261645    0.0127629    0.0408529

In [17]:
V, rdf = vcov(model, :HC2)
V

4×4 Array{Float64,2}:
  0.0527369  -0.023502    -0.0137535   -0.0558554
 -0.023502    0.0270918   -0.00664753   0.0336205
 -0.0137535  -0.00664753   0.0299603    0.017588 
 -0.0558554   0.0336205    0.017588     0.0543496

In [18]:
V, rdf = vcov(model, :HC3)
V

4×4 Array{Float64,2}:
  0.070778   -0.0300476   -0.019677    -0.075106 
 -0.0300476   0.0333185   -0.00836899   0.0435113
 -0.019677   -0.00836899   0.0391379    0.0246247
 -0.075106    0.0435113    0.0246247    0.0729389